In [1]:
!pip install transformers

In [2]:
import transformers
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

# Load the training data
with open("modified_faq_data.json", "r") as f:
    training_data = json.load(f)


In [3]:
# Convert the training data to a format that DialoGPT can understand
training_inputs = []
for item in training_data:
  training_inputs.append(item['question'])
  training_inputs.append(item['answer'])

training_inputs[0:4]

['What is the difference between a Commercial Representative and a Distributor?',
 'A MANE Commercial Representative is appointed to assist for the promotion and sale of MANE’s products (fragrances/ flavours/ ingredients), within a limited territory (country, region, city…) for a specific market (fragrance/ flavour for the food/ cosmetic/ personal care/ home care, and less frequently, in fine fragrance, pharmaceutical, tobacco, pet food, and oral care industries), and grow MANE’s direct sales accordingly. A MANE Distributor is appointed to resell “as is” some of MANE’s products (fragrances/ flavours/ ingredients), within a limited territory (country, region, city…) for a specific market (fragrance/ flavour for the food/ cosmetic/ personal care/ home care, and less frequently, in fine fragrance, pharmaceutical, tobacco, pet food, and oral care industries) to its own customers.',
 'What is the difference between a Commercial Representative and an Agent?',
 'An Agent is entitled to negoti

In [ ]:
# Tokenize the training data
tokenizer.pad_token = tokenizer.eos_token
encoded_inputs = tokenizer(training_inputs, truncation=True, padding=True, return_tensors="pt")

# Train the DialoGPT model
model.train()
batch_size = 4

for epoch in range(3):
    for i in range(0, len(training_inputs), batch_size):
        input_ids = encoded_inputs["input_ids"][i:i + batch_size]
        attention_mask = encoded_inputs["attention_mask"][i:i + batch_size]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)

